In [ ]:
import pandas as pd
import numpy as np 
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load data from ../Dataset

In [ ]:
df = pd.read_csv('../Dataset/df_processed.csv')

In [ ]:
df.head()

# Split data

In [ ]:
# Create label and features
X = df.drop('Grade', axis=1)
y = df['Grade']

In [ ]:
# Create train, test, and validation data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Train a catboost classifier

In [ ]:
# Identify categorical features
cat_features = list(range(0, X_train.shape[1]))

# Create the model
model = CatBoostClassifier(
    iterations=1000, 
    depth=7, 
    cat_features=cat_features, 
    verbose=200,
    early_stopping_rounds=50
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val))

# Cross validation

In [ ]:
# Define the pool
pool = Pool(X_train, y_train, cat_features=cat_features)

# Perform cross-validation
cv_scores = cv(pool, model.get_params(), fold_count=5, verbose=200)